In [3]:
import sys
sys.path.append("/home/hugo/projetos-doutorado/Appliance-Energy-Prediction")
import pandas as pd
from pca_fts.PcaWeightedMVFTS import PcaWeightedMVFTS
from pyFTS.benchmarks import Measures
import matplotlib.pyplot as plt
from pyFTS.common import Util
import datetime
import statistics
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Aux functions

In [4]:
def sample_first_prows(data, perc=0.75):
    return data.head(int(len(data)*(perc)))

In [5]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [6]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

## Dataset Energy Appliance

In [7]:
filename = '/home/hugo/projetos-doutorado/Appliance-Energy-Prediction/data/energydata_complete.csv'
data = pd.read_csv(filename)
data.pop('date')
data.pop('rv1')
data.pop('rv2')
data.dropna(inplace=True)
data.head()

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3
1,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2
2,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1
3,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0
4,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9


In [8]:
values = data.values
values

array([[ 60.        ,  30.        ,  19.89      , ...,   7.        ,
         63.        ,   5.3       ],
       [ 60.        ,  30.        ,  19.89      , ...,   6.66666667,
         59.16666667,   5.2       ],
       [ 50.        ,  30.        ,  19.89      , ...,   6.33333333,
         55.33333333,   5.1       ],
       ...,
       [270.        ,  10.        ,  25.5       , ...,   3.66666667,
         25.33333333,  13.26666667],
       [420.        ,  10.        ,  25.5       , ...,   3.83333333,
         26.16666667,  13.23333333],
       [430.        ,  10.        ,  25.5       , ...,   4.        ,
         27.        ,  13.2       ]])

### Dataset Energy Appliance: columns

In [9]:
cols = data.columns
list_cols = list(cols)
list_cols.append('Appliances_t+1')
print(list_cols)

['Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint', 'Appliances_t+1']


### Dataset Energy Appliance: Preprocessing, Lag = 1, t+1

In [10]:
# frame as supervised learning
reframed = series_to_supervised(values, 1, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]], axis=1, inplace=True)
#reframed.drop(reframed.columns[[0,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51]], axis=1, inplace=True)
reframed.head()

,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var6(t-1),var7(t-1),var8(t-1),var9(t-1),var10(t-1),...,var18(t-1),var19(t-1),var20(t-1),var21(t-1),var22(t-1),var23(t-1),var24(t-1),var25(t-1),var26(t-1),var1(t)
1,60.0,30.0,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,...,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,60.0
2,60.0,30.0,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,...,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,50.0
3,50.0,30.0,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,...,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,50.0
4,50.0,40.0,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,...,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,60.0
5,60.0,40.0,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,...,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,50.0


In [11]:
data = pd.DataFrame(reframed.values,columns=list_cols)
data = clean_dataset(data)
data

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,Appliances_t+1
0,60.0,30.0,19.890000,47.596667,19.200000,44.790000,19.790000,44.730000,19.000000,45.566667,...,48.900000,17.033333,45.5300,6.600000,733.5,92.000000,7.000000,63.000000,5.300000,60.0
1,60.0,30.0,19.890000,46.693333,19.200000,44.722500,19.790000,44.790000,19.000000,45.992500,...,48.863333,17.066667,45.5600,6.483333,733.6,92.000000,6.666667,59.166667,5.200000,50.0
2,50.0,30.0,19.890000,46.300000,19.200000,44.626667,19.790000,44.933333,18.926667,45.890000,...,48.730000,17.000000,45.5000,6.366667,733.7,92.000000,6.333333,55.333333,5.100000,50.0
3,50.0,40.0,19.890000,46.066667,19.200000,44.590000,19.790000,45.000000,18.890000,45.723333,...,48.590000,17.000000,45.4000,6.250000,733.8,92.000000,6.000000,51.500000,5.000000,60.0
4,60.0,40.0,19.890000,46.333333,19.200000,44.530000,19.790000,45.000000,18.890000,45.530000,...,48.590000,17.000000,45.4000,6.133333,733.9,92.000000,5.666667,47.666667,4.900000,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19729,90.0,0.0,25.533333,46.860000,25.978000,42.534000,27.323333,41.090000,24.700000,45.626667,...,50.445000,23.200000,46.7450,22.866667,755.2,55.333333,3.166667,22.833333,13.366667,100.0
19730,100.0,0.0,25.566667,46.560000,25.890000,42.025714,27.200000,41.163333,24.700000,45.590000,...,50.074000,23.200000,46.7900,22.733333,755.2,55.666667,3.333333,23.666667,13.333333,90.0
19731,90.0,0.0,25.500000,46.500000,25.754000,42.080000,27.133333,41.223333,24.700000,45.590000,...,49.790000,23.200000,46.7900,22.600000,755.2,56.000000,3.500000,24.500000,13.300000,270.0
19732,270.0,10.0,25.500000,46.596667,25.628571,42.768571,27.050000,41.690000,24.700000,45.730000,...,49.660000,23.200000,46.7900,22.466667,755.2,56.333333,3.666667,25.333333,13.266667,420.0


## Model: PcaWeightedMVFTS - PCA-MVFTS

In [12]:
pca_wmvfts = PcaWeightedMVFTS(n_components = 2,
                       endogen_variable = 'Appliances_t+1',
                       n_part = 50)

In [19]:
def sliding_window_pca(data,n_windows,train_size,dim,fs):

    result = {
         "window": [],
         "rmse": [],
         "mape": [],
         "smape": []
    }

    tam = len(data)
    n_windows = 30
    windows_length = math.floor(tam / n_windows)
    for ct, ttrain, ttest in Util.sliding_window(data, windows_length, 0.75, inc=1):
        if len(ttest) > 0:

#           print('-' * 20)
#           print(f'training window {(ct)}')
            pca_wmvfts = PcaWeightedMVFTS(n_components = dim,
                       endogen_variable = 'Appliances_t+1',
                       n_part = fs)
            model, pca_reduced_train = pca_wmvfts.run_train_model(ttrain)
            forecast, forecast_self, pca_reduced_test = pca_wmvfts.run_test_model(model,ttest)

            #print("[{0: %H:%M:%S}]".format(datetime.datetime.now()) + f" getting statistics")
            rmse = Measures.rmse(pca_reduced_test['Appliances_t+1'],forecast)
            mape = Measures.mape(pca_reduced_test['Appliances_t+1'],forecast)
            smape = Measures.smape(pca_reduced_test['Appliances_t+1'],forecast)

            # Grafico de comparação Original x Previsto para cada janela
#             fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[15, 3])
#             ax.plot(pca_reduced_test['Appliances_t+1'], label='Original')
#             ax.plot(forecast, label='Forecast')
#             handles, labels = ax.get_legend_handles_labels()
#             lgd = ax.legend(handles, labels, loc=2, bbox_to_anchor=(1, 1))
#             plt.show()

            result["rmse"].append(rmse)
            result["mape"].append(mape)
            result["smape"].append(smape)
            result["window"].append(ct)
        
    measures = pd.DataFrame(result)
    return measures



In [21]:
dimensions = [2,3,4,5,6]
fuzzy_sets = [10,20,30,40,50]

final_result = {
    "dimensions": [],
    "fuzzy_sets": [],
    "rmse": [],
     "mape": [],
     "smape": []
}

n_windows = 30
train_size = 0.75

for dim in dimensions:
    for fs in fuzzy_sets:
        measures = sliding_window_pca(data,n_windows,train_size,dim,fs)
        final_result["dimensions"].append(dim)
        final_result["fuzzy_sets"].append(fs)
        
        rmse = statistics.mean(measures['rmse'])
        mape = statistics.mean(measures['mape'])
        smape = statistics.mean(measures['smape'])
        
        final_result["rmse"].append(rmse)
        final_result["mape"].append(mape)
        final_result["smape"].append(smape)
        
        print(f'Results: {(dim,fs,rmse,mape,smape)}')
        
        
final_measures_pca = pd.DataFrame(final_result) 

print("Statistics PCA-WMVFTS (test): ")
final_measures_pca

Results: (2, 10, 29.820903424921962, 30.248957775796892, 11.769431726620779)
Results: (2, 20, 16.15732250433916, 10.551936004656174, 4.559606073254191)
Results: (2, 30, 9.359260662326255, 5.3001750602094635, 2.287914779351237)
Results: (2, 40, 7.665963011771226, 3.2039588486269217, 1.3150062057692338)
Results: (2, 50, 5.504215355498991, 1.799567798444724, 0.7307637903377124)
Results: (3, 10, 18.629599316425292, 14.570939022342575, 5.544477411854706)
Results: (3, 20, 4.9752270296055086, 2.4344760139044426, 1.0907417792458087)
Results: (3, 30, 1.4707999206814881, 0.6117317588960464, 0.291699101606888)
Results: (3, 40, 0.8488735458484737, 0.2787671801856652, 0.1330103168212365)
Results: (3, 50, 0.5045269907071805, 0.12581366731492935, 0.05877405679782321)
Results: (4, 10, 7.495889259598014, 4.748267942595504, 1.9901200725995212)
Results: (4, 20, 0.9543337977044175, 0.41141893008224767, 0.196054596603099)
Results: (4, 30, 0.2940432484651523, 0.1056498946467261, 0.047695729189737406)
Result

,dimensions,fuzzy_sets,rmse,mape,smape
0,2,10,29.820903,30.248958,11.769432
1,2,20,16.157323,10.551936,4.559606
2,2,30,9.359261,5.300175,2.287915
3,2,40,7.665963,3.203959,1.315006
4,2,50,5.504215,1.799568,0.730764
5,3,10,18.629599,14.570939,5.544477
6,3,20,4.975227,2.434476,1.090742
7,3,30,1.470800,0.611732,0.291699
8,3,40,0.848874,0.278767,0.133010
9,3,50,0.504527,0.125814,0.058774


In [22]:
final_measures_pca

,dimensions,fuzzy_sets,rmse,mape,smape
0,2,10,29.820903,30.248958,11.769432
1,2,20,16.157323,10.551936,4.559606
2,2,30,9.359261,5.300175,2.287915
3,2,40,7.665963,3.203959,1.315006
4,2,50,5.504215,1.799568,0.730764
5,3,10,18.629599,14.570939,5.544477
6,3,20,4.975227,2.434476,1.090742
7,3,30,1.470800,0.611732,0.291699
8,3,40,0.848874,0.278767,0.133010
9,3,50,0.504527,0.125814,0.058774
